**importing the required libraries**

In [1]:
import cv2
import numpy as np
import face_recognition
import tensorflow as tf
from tensorflow import keras  # Accessing Keras through TensorFlow
from tensorflow.keras.models import model_from_json 
from tensorflow.keras.preprocessing import image

**capture the video from default camera**

In [2]:
webcam_video_stream = cv2.VideoCapture(0)

**Checking if the camera opened successfully**

In [3]:
if not webcam_video_stream.isOpened():
    print("Error: Could not open video stream.  Check camera index.")
    exit()

**Load the model and load the weights**

In [4]:
face_exp_model = model_from_json(open("dataset\\facial_expression_model_structure.json","r").read())
face_exp_model.load_weights('dataset\\facial_expression_model_weights.h5')

**declare the emotions label**

In [5]:
emotions_label = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

**initialize the array variable to hold all face locations in the frame**

In [6]:
all_face_locations = []

**loop through every frame in the video**

In [7]:
while True:
    # get the current frame from the video stream as an image
    ret, current_frame = webcam_video_stream.read()

    # Check if the frame was successfully captured
    if not ret or current_frame is None:
        print("Failed to capture image from webcam")
        break

    # resize the current frame to 1/4 size to process faster
    current_frame_small = cv2.resize(current_frame, (0, 0), fx=0.25, fy=0.25)
    # detect all faces in the image
    # arguments are image, no_of_times_to_upsample, model
    all_face_locations = face_recognition.face_locations(current_frame_small, number_of_times_to_upsample=2,
                                                        model='hog')

    # looping through the face locations
    for index, current_face_location in enumerate(all_face_locations):
        # splitting the tuple to get the four position values of current face
        top_pos, right_pos, bottom_pos, left_pos = current_face_location
        # change the position maginitude to fit the actual size video frame
        top_pos = top_pos * 4
        right_pos = right_pos * 4
        bottom_pos = bottom_pos * 4
        left_pos = left_pos * 4
        # printing the location of current face
        print('Found face {} at top:{},right:{},bottom:{},left:{}'.format(index + 1, top_pos, right_pos, bottom_pos,
                                                                            left_pos))

        # Extract the face from the frame, blur it, paste it back to the frame
        # slicing the current face from main image
        current_face_image = current_frame[top_pos:bottom_pos, left_pos:right_pos]

        # draw rectangle around the face detected
        cv2.rectangle(current_frame, (left_pos, top_pos), (right_pos, bottom_pos), (0, 0, 255), 2)

        # preprocess input, convert it to an image like as the data in dataset
        # convert to grayscale
        current_face_image = cv2.cvtColor(current_face_image, cv2.COLOR_BGR2GRAY)
        # resize to 48x48 px size
        current_face_image = cv2.resize(current_face_image, (48, 48))
        # convert the PIL image into a 3d numpy array
        img_pixels = image.img_to_array(current_face_image)
        # expand the shape of an array into single row multiple columns
        img_pixels = np.expand_dims(img_pixels, axis=0)
        # pixels are in range of [0, 255]. normalize all pixels in scale of [0, 1]
        img_pixels /= 255

        # do prodiction using model, get the prediction values for all 7 expressions
        exp_predictions = face_exp_model.predict(img_pixels)
        # find max indexed prediction value (0 till 7)
        max_index = np.argmax(exp_predictions[0])
        # get corresponding lable from emotions_label
        emotion_label = emotions_label[max_index]

        # display the name as text in the image
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(current_frame, emotion_label, (left_pos, bottom_pos), font, 0.5, (255, 255, 255), 1)

    # showing the current face with rectangle drawn
    cv2.imshow("Webcam Video", current_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the stream and cam
# close all opencv windows open
webcam_video_stream.release()
cv2.destroyAllWindows()

Found face 1 at top:132,right:512,bottom:348,left:296
1/1 [==============================] - 0s 155ms/step
Found face 1 at top:132,right:488,bottom:348,left:272
1/1 [==============================] - 0s 19ms/step
Found face 1 at top:132,right:488,bottom:348,left:272
1/1 [==============================] - 0s 22ms/step
Found face 1 at top:132,right:488,bottom:348,left:272
1/1 [==============================] - 0s 19ms/step
Found face 1 at top:128,right:464,bottom:308,left:284
1/1 [==============================] - 0s 22ms/step
Found face 1 at top:128,right:444,bottom:308,left:264
1/1 [==============================] - 0s 20ms/step
Found face 1 at top:108,right:464,bottom:324,left:248
1/1 [==============================] - 0s 19ms/step
Found face 1 at top:108,right:464,bottom:324,left:248
1/1 [==============================] - 0s 23ms/step
Found face 1 at top:128,right:444,bottom:308,left:264
1/1 [==============================] - 0s 21ms/step
Found face 1 at top:128,right:444,bottom:308,